In [1]:
import pandas as pd
import numpy as np

In [2]:
file_train=r"I:\data scientist using python\project1\P1 Data\Consumer_Complaints_train.csv"
cd=pd.read_csv(file_train)
file_test=r"I:\data scientist using python\project1\P1 Data\Consumer_Complaints_test_share.csv"
test=pd.read_csv(file_test)

## Data Cleaning

In [3]:
# Col: Date received, Date sent to company
for col in ['Date received','Date sent to company']:
    cd[col]=pd.to_datetime(cd[col],infer_datetime_format=True)
    test[col]=pd.to_datetime(test[col],infer_datetime_format=True)
cd['day_diff']=pd.to_numeric((cd['Date sent to company']-cd['Date received'])/86400000000000)
test['day_diff']=pd.to_numeric((test['Date sent to company']-test['Date received'])/86400000000000)

for col in ['Date received','Date sent to company']:
    del cd[col]
    del test[col]

#from datetime import datetime
#date_format="%Y-%m-%d"
#cd['date_diff']=0
#for i in range(0,cd.shape[0]):
 #   a = datetime.strptime(cd['Date sent to company'][i], date_format)
  #  b = datetime.strptime(cd['Date received'][i], date_format)
   # delta = a - b
    #days=delta.days
    #cd['date_diff'][i]=days
#del cd['Date sent to company']
#del cd['Date received']

# Col: ZIP code
del cd['ZIP code']
del test['ZIP code']

# product, sub-product
k1=pd.get_dummies(cd['Product'],drop_first=True,prefix='Product')
cd=pd.concat([cd,k1],axis=1)
del cd['Product']

k2=pd.get_dummies(test['Product'],drop_first=True,prefix='Product')
test=pd.concat([test,k2],axis=1)
del test['Product']

cd['Sub-product']=np.where(cd['Sub-product'].isnull(),0,1)
test['Sub-product']=np.where(test['Sub-product'].isnull(),0,1)

# col: Issue,sub-issue
freq=cd['Issue'].value_counts()
k=freq.index[freq>=9000][:-1]
for cat in k:
    name='Issue_'+cat.replace(',','_').replace(' ','_')
    cd[name]=(cd['Issue']==cat).astype('int')
    test[name]=(test['Issue']==cat).astype('int')
del cd['Issue']
del test['Issue']

cd['Sub-issue']=np.where(cd['Sub-issue'].isnull(),0,1)
test['Sub-issue']=np.where(test['Sub-issue'].isnull(),0,1)
del cd['Sub-issue'],test['Sub-issue']

# Col: Consumer disputed
cd['Consumer_disputed']=cd['Consumer disputed?'].map({'Yes':1,
                                                    'No':0})
del cd['Consumer disputed?']

# Col: Timely Response
cd['Timely_response']=cd['Timely response?'].map({'Yes':1,'No':0})
test['Timely_response']=test['Timely response?'].map({'Yes':1,'No':0})

del cd['Timely response?'],test['Timely response?']

# Col: Complaint ID
del cd['Complaint ID'], test['Complaint ID']
# Col: Company
freq=cd['Company'].value_counts()
k=freq.index[freq>=10000][:-1]
for cat in k:
    name='Company_'+cat.replace(',','_').replace(' ','_')
    cd[name]=(cd['Company']==cat).astype('int')
    test[name]=(test['Company']==cat).astype('int')
del cd['Company']
del test['Company']

# Col:Tags
cd['Tags']=np.where(cd['Tags'].isnull(),0,1)
test['Tags']=np.where(test['Tags'].isnull(),0,1)     

# Col: State
freq=cd['State'].value_counts()
k=freq.index[freq>10000]
for val in k:
    name='State_'+val
    cd[name]=(cd['State']==val).astype(int)
    test[name]=(test['State']==val).astype(int)
del cd['State'],test['State']

# Col: Company public response
k1=pd.get_dummies(cd['Company public response'],drop_first=True,prefix='Company_public_response')
cd=pd.concat([cd,k1],axis=1)
del cd['Company public response']

k2=pd.get_dummies(test['Company public response'],drop_first=True,prefix='Company_public_response')
test=pd.concat([test,k2],axis=1)
del test['Company public response']

# Col:Company response to consumer, Submitted via
for i in ['Company response to consumer','Submitted via']:
    name=i.replace(' ','_')
    k1=pd.get_dummies(cd[i],drop_first=True,prefix=name)
    cd=pd.concat([cd,k1],axis=1)

    k2=pd.get_dummies(test[i],drop_first=True,prefix=name)
    test=pd.concat([test,k2],axis=1)
    
    del test[i]
    del cd[i]

# Col: Consumer_consent_provided_
freq=cd['Consumer consent provided?'].value_counts()
k=freq.index
for val in k:
    name='Consumer_consent_provided_'+val.replace(' ','_')
    cd[name]=(cd['Consumer consent provided?']==val).astype(int)
    test[name]=(test['Consumer consent provided?']==val).astype(int)
del cd['Consumer consent provided?'],test['Consumer consent provided?']

# Col: Consumer_complaint_narrative
cd['Consumer_complaint_narrative']=np.where(cd['Consumer complaint narrative'].isnull(),0,1)
test['Consumer_complaint_narrative']=np.where(test['Consumer complaint narrative'].isnull(),0,1)

del cd['Consumer complaint narrative'],test['Consumer complaint narrative']

## Logistic regression

In [9]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import roc_auc_score

In [10]:
cd_train,cd_test=train_test_split(cd,test_size=0.2,random_state=3)

In [11]:
cd_train.shape

(382736, 76)

In [12]:
cd_test.shape

(95685, 76)

In [13]:
x_cd_train=cd_train.drop('Consumer_disputed',axis=1)
y_cd_train=cd_train['Consumer_disputed']
x_cd_test=cd_test.drop('Consumer_disputed',axis=1)
y_cd_test=cd_test['Consumer_disputed']

In [14]:
x_cd_train.shape

(382736, 75)

In [15]:
logreg=LogisticRegression(class_weight='balanced')
logreg.fit(x_cd_train,y_cd_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [16]:
p=logreg.predict_proba(x_cd_train)[:,1]
roc_auc_score(y_cd_train,p)

0.6162343759859666

In [17]:
p=logreg.predict_proba(x_cd_test)[:,1]
roc_auc_score(y_cd_test,p)

0.6171930800188312

In [ ]:
# Tuning Hyperparameter

#iteration1
C1=np.linspace(1,100,100)
roc_auc=[]
for alpha in C1:
    logreg=LogisticRegression(class_weight='balanced',C=alpha)
    logreg.fit(x_cd_train,y_cd_train)
    p=logreg.predict_proba(x_cd_train)[:,1]
    roc=roc_auc_score(y_cd_train,p)
    roc_auc.append(roc)

In [ ]:
alpha=C1[roc_auc==max(roc_auc)]

In [18]:
logreg=LogisticRegression(class_weight='balanced',C=15)
logreg.fit(x_cd_train,y_cd_train)

LogisticRegression(C=15, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [19]:
p=logreg.predict_proba(x_cd_test)[:,1]
roc_auc_score(y_cd_test,p)

0.6171895367346574

In [ ]:
## Confusion Matrix analysis
KS_all=[]
Fbeta=[]
Accuracy=[]
Precision=[]
cutoffs=np.linspace(0.01,0.99,99)
p=logreg.predict_proba(x_cd_train)[:,1]
real=y_cd_train
for cutoff in cutoffs:
    predicted=(p>cutoff).astype(int)
    
    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    P=TP+FN
    N=FP+TN
    KS=(TP/P) - (FP/N)
    prec= TP/(TP+FP)
    Recall=TP/P
    accu=(TP+TN)/(P+N)
    B=5
    Fb=((1+B*B)*prec* Recall)/(B*B*prec+Recall)
    KS_all.append(KS)
    Fbeta.append(Fb)
    Accuracy.append(accu)
    Precision.append(prec)
print('Cutoff for max accuracy of ',max(Accuracy)*100,'% is ',max(cutoffs[Accuracy==max(Accuracy)]))
print('Cutoff for max Precision of ',max(Precision)*100,'% is ',max(cutoffs[Precision==max(Precision)]))
print('Cutoff for max Fbeta of ',max(Fbeta),' is ',max(cutoffs[Fbeta==max(Fbeta)]))
print('Cutoff for max KS_all of ',max(KS_all),' is ',max(cutoffs[KS_all==max(KS_all)]))


In [ ]:
p=(logreg.predict_proba(x_cd_train)[:,1]>0.5).astype(int)

In [ ]:
roc_auc_score(y_cd_train,p)

In [ ]:
p[p==max(p)]

## Decision Tree

In [17]:
from sklearn.metrics import accuracy_score
from sklearn import tree

x_cd_train.reset_index(drop=True,inplace=True)
y_cd_train.reset_index(drop=True,inplace=True)

x_cd_test.reset_index(drop=True,inplace=True)
y_cd_test.reset_index(drop=True,inplace=True)

dtree=tree.DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=20)
dtree.fit(x_cd_train,y_cd_train)
p=dtree.predict(x_cd_test)
pb=dtree.predict_proba(x_cd_test)[:,1]
print('-------------------------------')
print('ROC AUC Score for decision tree is ',round(roc_auc_score(y_cd_test,pb),2))

# Creating Tree
#dotfile=open("tree_prediction_sales1.dot",'w')

#tree.export_graphviz(dtree,out_file=dotfile,
 #                   feature_names=x_train.columns,
  #                  class_names=['0','1'],
   #                 proportion=True)
#dotfile.close()

# plot tree in www.webgraphviz.com using dot file generated
score=accuracy_score(y_cd_test,p)
print('-------------------------------')
print('Accuracy Using Decision tree is ', round(score*100,2),'%')
print('-------------------------------')

-------------------------------
ROC AUC Score for decision tree is  0.61
-------------------------------
Accuracy Using Decision tree is  78.8 %
-------------------------------


## Random Forest

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Defining parameters of random forest
parameters={'n_estimators':[10,50,100,300,500],
            'max_features':[5,10,15,20],
            'max_depth':[3,5,10,20,30,None],
            'min_samples_split':[2,5,10,15,20],
            'min_samples_leaf':[1,5,10,15,20],
            'criterion':['entropy','gini'],
            'bootstrap':[True,False] 
           }
# Total Combinations possible 5*4*6*5*5*2*2=120000

# Defining class
clf=RandomForestClassifier(verbose=1,n_jobs=-1)

# Searching 20 random models from 3072 combinations
# Best way is to execute all the combinations and then look for best. But in this computation effort is enormous
n_iter_search=5
random_search=RandomizedSearchCV(clf,param_distributions=parameters,n_iter=n_iter_search,scoring='roc_auc',cv=3,
                                 return_train_score=True)
random_search.fit(x_cd_train,y_cd_train)

# Function for report generation
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:.3f} (std: {1:.3f})'.format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
# Calling the function report to get top performing model... n=3
print('Best Performing Models are :',report(random_search.cv_results_,3))

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   36.8s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   51.5s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   47.4s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs

[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    8.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   15.8s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   17.9s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.9min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    8.8s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    9.9s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | e

Model with rank: 1
Mean validation score: 0.643 (std: 0.002)
Parameters: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 15, 'max_features': 10, 'max_depth': 30, 'criterion': 'gini', 'bootstrap': False}

Model with rank: 2
Mean validation score: 0.642 (std: 0.001)
Parameters: {'n_estimators': 500, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 5, 'max_depth': 30, 'criterion': 'gini', 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.640 (std: 0.002)
Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 15, 'max_depth': 30, 'criterion': 'gini', 'bootstrap': True}

Best Performing Models are : None


In [17]:
# Best model parameters
random_search.cv_results_['params'][random_search.best_index_]

{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 15,
 'max_features': 10,
 'max_depth': 30,
 'criterion': 'gini',
 'bootstrap': False}

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
# Defining Class for above obtained parameter combinations
rf=RandomForestClassifier(n_estimators=100,min_samples_split=5,max_depth=30,max_features=15,bootstrap=True,
                         criterion='gini',min_samples_leaf=5,class_weight='balanced',verbose=1)
# Fitting model to rain data
rf.fit(x_cd_train,y_cd_train)

# Prediction on test data
p_rf=rf.predict_proba(x_cd_test)[:,1]

# Performance metrics : ROC score
print('-------------------------------')
print('ROC AUC score for Random Forest model is', roc_auc_score(y_cd_test,p_rf))
print('-------------------------------')
# Performance metrics : Accuracy
from sklearn.metrics import accuracy_score
prf=rf.predict(x_cd_test)
score1=accuracy_score(y_cd_test,prf)
print('Accuracy Using Random Forest is ', round(score1*100,2),'%')
print('-------------------------------')

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.6s finished


-------------------------------
ROC AUC score for Random Forest model is 0.6227696026352453
-------------------------------
Accuracy Using Random Forest is  58.25 %
-------------------------------


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.7s finished


In [25]:
# Important features Ranking
importances = rf.feature_importances_

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(x_cd_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, list(x_cd_train.columns)[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature day_diff (0.176751)
2. feature Company_response_to_consumer_Closed with explanation (0.050136)
3. feature Submitted_via_Web (0.042918)
4. feature Company_response_to_consumer_Closed with non-monetary relief (0.042771)
5. feature Company_response_to_consumer_Closed with monetary relief (0.036148)
6. feature Tags (0.028402)
7. feature State_CA (0.022186)
8. feature State_FL (0.019874)
9. feature State_TX (0.019250)
10. feature State_NY (0.017792)
11. feature Product_Mortgage (0.017400)
12. feature Company_Bank_of_America (0.017059)
13. feature Company_public_response_Company chooses not to provide a public response (0.016508)
14. feature Submitted_via_Referral (0.016458)
15. feature Issue_Incorrect_information_on_credit_report (0.015783)
16. feature Product_Debt collection (0.015353)
17. feature Consumer_consent_provided_Consent_not_provided (0.013815)
18. feature State_GA (0.013736)
19. feature State_NJ (0.013626)
20. feature Company_public_response_Company h

## Neural Network

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold

parameters={'hidden_layer_sizes':[(10,5),(5,10),(15,10),(10,15),(20,10),(5,10,15),(15,10,5),(20,15,10),(10,15,20)],
           'learning_rate':['constant', 'invscaling', 'adaptive'],
           'learning_rate_init':[0.01,0.1,0.001],
           'activation':['identity', 'logistic', 'tanh', 'relu']}

NNclf=MLPClassifier()
rsv=RandomizedSearchCV(NNclf,n_iter=5,cv=3,param_distributions=parameters,n_jobs=-1,verbose=20)
rsv.fit(x_cd_train,y_cd_train)


# Function for report generation
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:.3f} (std: {1:.3f})'.format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
# Calling the function report to get top performing model... n=3
report(rsv.cv_results_,3)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   9 out of  15 | elapsed:  2.5min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  2.7min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  11 out of  15 | elapsed:  2.9min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  3.4min remaining:   50.9s
[Parallel(n_jobs=-1)]: Done  13 out of  15 | elapsed:  3.7min remaining:   33.6s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.0min remaining:    0.0s
[Parallel(n_jobs=-

Model with rank: 1
Mean validation score: 0.788 (std: 0.000)
Parameters: {'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 15, 20), 'activation': 'tanh'}

Model with rank: 2
Mean validation score: 0.788 (std: 0.000)
Parameters: {'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (20, 10), 'activation': 'identity'}

Model with rank: 2
Mean validation score: 0.788 (std: 0.000)
Parameters: {'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 15, 20), 'activation': 'tanh'}

Model with rank: 2
Mean validation score: 0.788 (std: 0.000)
Parameters: {'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (20, 15, 10), 'activation': 'logistic'}

Model with rank: 2
Mean validation score: 0.788 (std: 0.000)
Parameters: {'learning_rate_init': 0.01, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 15), 'activation': 'identity'}



In [30]:
NNclf=MLPClassifier(hidden_layer_sizes=(10,15,20),learning_rate='adaptive',activation='tanh',learning_rate_init=0.01)
NNclf.fit(x_cd_train,y_cd_train)
pred_test=NNclf.predict_proba(x_cd_test)[:,1]
roc_auc_score(y_cd_test,pred_test)

0.6219847913404667

## AdaBoost || XGBoost

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV

In [21]:
cd_train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [22]:
x_cd_train=cd_train.drop('Consumer_disputed',axis=1)
y_cd_train=cd_train['Consumer_disputed']
x_cd_test=cd_test.drop('Consumer_disputed',axis=1)
y_cd_test=cd_test['Consumer_disputed']

In [14]:
base_clf=tree.DecisionTreeClassifier(max_leaf_nodes=10,class_weight='balanced')
adb_params={'n_estimators':[50,100,200,500,700],
           'learning_rate': [0.01,.1,1]
           }
adb=AdaBoostClassifier(base_estimator=base_clf)

n_iter_search=5
random_search=RandomizedSearchCV(adb,param_distributions=adb_params,n_iter=n_iter_search,scoring='roc_auc',cv=3,
                                 return_train_score=True,verbose=2,n_jobs=-1)
random_search.fit(x_cd_train,y_cd_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 73.6min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
          fit_params=None, iid=True, n_iter=5, n_jobs=-1,
          param_distributions={'n_estimators': [50, 100, 200, 500, 700], 'learning_rate': [0.01, 0.1, 1]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=2)

In [15]:
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:.3f} (std: {1:.3f})'.format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
# Calling the function report to get top performing model... n=3
print('Best Performing Models are :',report(random_search.cv_results_,3))

Model with rank: 1
Mean validation score: 0.620 (std: 0.001)
Parameters: {'n_estimators': 500, 'learning_rate': 0.01}

Model with rank: 2
Mean validation score: 0.620 (std: 0.002)
Parameters: {'n_estimators': 50, 'learning_rate': 1}

Model with rank: 3
Mean validation score: 0.618 (std: 0.002)
Parameters: {'n_estimators': 100, 'learning_rate': 1}

Best Performing Models are : None


In [ ]:
base_clf=tree.DecisionTreeClassifier(max_leaf_nodes=10,class_weight='balanced')
adb_best=AdaBoostClassifier(base_estimator=base_clf,learning_rate=0.01,n_estimators=500)
adb_best.fit(x_cd_train,y_cd_train)
p=adb_best.predict_proba(x_cd_test)[:,1]
roc_auc_score(y_cd_test,p)

In [17]:
roc_auc_score(y_cd_test,p)

0.620490048740005

In [23]:
param_dist = {
    
              "max_depth": [3,4,5,6,7],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000],
    'scale_pos_weight':[2,3,4,5,6,7,8,9]
    
              }
clf=XGBClassifier(objective='binary:logistic')

n_iter=10

random_search=RandomizedSearchCV(clf,n_jobs=-1,verbose=20,cv=3,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist)

In [24]:
random_search.fit(x_cd_train,y_cd_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 36.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 38.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 39.6min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 39.7min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 81.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 88.0min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 95.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 96.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 98.3min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 104.9min
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed: 112.9min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed: 113.7min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 115.5min
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed: 123.7min
[Parallel(n_jobs=-1)]: Done  23 tasks      | elapsed: 131.8min
[P

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'max_depth': [3, 4, 5, 6, 7], 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5], 'min_child_weight': [4, 5, 6], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'reg_alpha': [1e-05, 0.01, 0.1, 1, 100], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'n_estimators': [100, 500, 700, 1000], 'scale_pos_weight': [2, 3, 4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scori

In [26]:
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:.3f} (std: {1:.3f})'.format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
report(random_search.cv_results_,3)

Model with rank: 1
Mean validation score: 0.626 (std: 0.001)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 6, 'reg_alpha': 0.1, 'n_estimators': 500, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.8}

Model with rank: 2
Mean validation score: 0.626 (std: 0.002)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 3, 'reg_alpha': 100, 'n_estimators': 1000, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.4, 'colsample_bytree': 0.8}

Model with rank: 3
Mean validation score: 0.626 (std: 0.001)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 2, 'reg_alpha': 1, 'n_estimators': 500, 'min_child_weight': 6, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.8}



In [ ]:
xgb_best=XGBClassifier(subsample=0.6,scale_pos_weight=6,reg_alpha=0.1,n_estimators=500,min_child_weight=6,
                       max_depth=4,learning_rate=0.05,gamma=0.1,colsample_bytree=0.8
                      )
xgb_best.fit(x_cd_train,y_cd_train)

p=xgb_best.predict_proba(x_cd_test)[:,1]
roc_auc_score(y_cd_test,p)

## Stacking

In [32]:
x_train=pd.DataFrame()
x_train['g1']=logreg.predict_proba(x_cd_train)[:,1]
x_train['g2']=dtree.predict_proba(x_cd_train)[:,1]
x_train['g3']=rf.predict_proba(x_cd_train)[:,1]
x_train['g4']=NNclf.predict_proba(x_cd_train)[:,1]

LR=LogisticRegression(class_weight='balanced')
LR.fit(x_train,y_cd_train)

x_test=pd.DataFrame()
x_test['g1']=logreg.predict_proba(x_cd_test)[:,1]
x_test['g2']=dtree.predict_proba(x_cd_test)[:,1]
x_test['g3']=rf.predict_proba(x_cd_test)[:,1]
x_test['g4']=NNclf.predict_proba(x_cd_test)[:,1]

p=LR.predict_proba(x_test)[:,1]
roc_auc_score(y_cd_test,p)

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   14.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.6s finished


0.5985766087482918

In [36]:
file_test=r"I:\data scientist using python\project1\P1 Data\Consumer_Complaints_test_share.csv"
test1=pd.read_csv(file_test)

In [47]:
submission=pd.DataFrame({'Complaint ID':test1['Complaint ID']})
submission['Consumer disputed?']=NNclf.predict(test)
submission['Consumer disputed?']=submission['Consumer disputed?'].map({1:'Yes',0:'No'})
submission['Consumer disputed?'].value_counts()

No     119580
Yes        26
Name: Consumer disputed?, dtype: int64

In [60]:
submission=pd.DataFrame({'Complaint ID':test1['Complaint ID']})
submission['Consumer disputed?']=rf.predict(test)
submission['Consumer disputed?']=submission['Consumer disputed?'].map({1:'Yes',0:'No'})
submission['Consumer disputed?'].value_counts()

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.2s finished


No     65206
Yes    54400
Name: Consumer disputed?, dtype: int64

In [56]:
y_cd_test.value_counts()

0    75397
1    20288
Name: Consumer_disputed, dtype: int64

In [49]:
submission=pd.DataFrame({'Complaint ID':test1['Complaint ID']})
submission['Consumer disputed?']=logreg.predict(test)
submission['Consumer disputed?']=submission['Consumer disputed?'].map({1:'Yes',0:'No'})
submission['Consumer disputed?'].value_counts()

No     61532
Yes    58074
Name: Consumer disputed?, dtype: int64

In [61]:
writer = pd.ExcelWriter('submission.xlsx')
submission.to_excel(writer,index=False)
writer.save()